### 

In [1]:
import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
from models import BookTable, TagTable, SentenceTable, BookTagTable, UserResponseTable
from connection import database_engine

# 데이터베이스 연결 설정
host = "localhost"
port = 5432
user = "postgres"
password = "1234"
database_name = "book_recommend"

engine = database_engine(host, port, user, password, database_name)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
json_file_path = '../../data/scraping/all_book_data_ver_cleaned.json'
with open(json_file_path, 'r', encoding='utf-8') as file:
  data = json.load(file)
  for book in data:      
    book_entry = BookTable(
                isbn=book["isbn13"],
                title=book["title"],
                publisher=book["publisher"],
                author=book["author"],
                image_url=book["img_url"],
                category='essay',
                publication_date = book["publication_date"]
            )
    try:
      # 데이터 삽입
      session.merge(book_entry)
      session.commit()
    except IntegrityError:
      # 중복된 키가 발생한 경우 롤백 후 처리
      session.rollback()
      print(f"중복된 ISBN: {book.isbn} - 삽입을 건너뜁니다.")
    except Exception as e:
        # 기타 예외 발생 시 롤백 및 에러 메시지 출력
        session.rollback()
        print(f"오류 발생: {e}")

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session

from database.crud import get_book_by_isbn
from database.schemas import BookSchema
from typing import List

# ✅ 특정 ISBN으로 도서 조회 API
@app.get("/books/{isbn}", response_model=BookSchema)
def read_book(isbn: str, db: Session = Depends(get_db)):
    book = get_book_by_isbn(db, isbn)
    if book is None:
        raise HTTPException(status_code=404, detail="해당 ISBN의 도서를 찾을 수 없습니다.")
    return book


In [2]:
from database.connection import get_db
db = get_db()

ModuleNotFoundError: No module named 'database.connection'; 'database' is not a package

In [ ]:

    db.query(BookTable).filter(BookTable.isbn == isbn).first()